In [16]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import re
import os
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# To download necessary resources:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/przemyslaw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/przemyslaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/przemyslaw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
posts = pd.read_csv(os.path.join('data', 'writers.stackexchange.com', 'Posts.csv'))

In [5]:
def clean_html(text):
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, '', text)
    text = text.replace("\n", " ").replace("\t", " ").replace("\\", "")
    return text

def clean_text(text):
    text = clean_html(text)
    text = text.lower()
    
    word_list = word_tokenize(text)
    word_list = [word.strip() for word in word_list if not all(letter in string.punctuation for letter in word)]
    word_list = ["<NUM>" if word.isdigit() else word for word in word_list]
    
    return " ".join(word_list)

In [6]:
texts = posts.Body[posts.Body.notna()]
texts = [clean_text(body) for body in texts]  # takes a few minuts to complete
texts

["i 've always wanted to start writing in a totally amateur way but whenever i want to start something i instantly get blocked having a lot of questions and doubts are there some resources on how to start becoming a writer i 'm thinking something with tips and easy exercises to get the ball rolling",
 'what kind of story is better suited for each point of view are there advantages or disadvantages inherent to them for example writing in the first person you are always following a character while in the third person you can jump between story lines',
 "i finished my novel and everyone i 've talked to says i need an agent how do i find one",
 'when writing a short-story to highlight a certain truth is it typically a better practice to construct a point to exemplify through the short-story in other words construct the characters plot and setting based on the point i want to portray instead of focusing on plot first although i need to make sure the plot is engaging is this a good practice'

In [7]:
cv = CountVectorizer(max_df=0.3, min_df=0.01, stop_words=list(stopwords.words('english')))
cv_X = cv.fit_transform(texts)
cv_X

<32130x1318 sparse matrix of type '<class 'numpy.int64'>'
	with 1462719 stored elements in Compressed Sparse Row format>

Jak czytamy w dokumentacji (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

Convert a collection of text documents to a matrix of token counts
This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

Przyjmuje ciąg stringów i metodą fit_transform zwraca macierz wystąpień poszczególnych tokenów.

max_df - część najczęściej występujących wyrazów, które chcemy wyciąć

min_df - część najrzadziej występujących wyrazów, które chcemy wyciąć

stopwords - lista stopwordów, które chcemy ignorować

In [8]:
counts = pd.DataFrame({
    'word': cv.get_feature_names(),
    'count': cv_X.toarray().sum(axis=0)
})
counts.nlargest(10, columns='count')

,word,count
161,character,20201
774,num,16017
1305,write,14389
123,book,14142
1261,want,13800
674,make,13789
1240,use,13323
827,people,13224
162,characters,12629
924,reader,12404


## LDA
Latent Dirichlet Allocation with online variational Bayes algorithm (http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)

n_componenets - liczba tematów, które chcemy przyporządkować danym

max_iter - maksymalna liczba iteracji (żeby się za długo nie liczyło)

learning_method - 'batch' lub 'online' (używane tylko w metodzie fit). Online jest szybsza, batch używa wszystkich danych treningowych w każdej iteracji EM do zaktualizowania components_.

Na obiekcie lda po wywołaniu metody fit będziemy mogli zobaczyć lda.components_, czyli tablicę wymiaru \[n_components, n_features\], gdzie element (i,j) reprezentuje jak często słowo j jest przypisywane do tematu i.

In [9]:
n_components = 3
lda = LatentDirichletAllocation(n_components = n_components, max_iter=5, learning_method='online', random_state=123)
lda.fit(cv_X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=3, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=123, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [32]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = 'Topic #%d: ' % topic_idx
        message += ' '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message + "\n")
    print()

In [33]:
print_top_words(lda, cv.get_feature_names(), 50)

Topic #0: write book work get read people good want time know find make books num think first something need way even writer go much many also well may writers really reading try lot might novel idea could ideas things better best written start take fiction going help say see someone published

Topic #1: character characters reader make way something people think person world could know want even first plot time things scene example good might need point much life readers also different may protagonist main two see get say real feel going really show stories well end tell without another thing hero said

Topic #2: num use name word using text words used example style page also book different sentence information source author may question chapter copyright work technical two language case paragraph need could names title first want number way might version software many etc questions specific pages line document original reference content article




Mozna poeksperymentować z liczbą topiców. Dla n_components=3 widzimy, że zerowy temat dotyczy głównie ludzi, którzy chcą zająć się pisaniem i publikować swoje opowieści, pierwszy dotyczy bardziej książek i ich treści natomiast drugi jest bardziej techniczny. 

Możemy też podejrzeć posty należące do tych tematów.

In [15]:
lda_features = lda.transform(cv_X[:10, ])
lda_features

array([[0.96725055, 0.01685198, 0.01589747],
       [0.02627304, 0.94285227, 0.03087469],
       [0.90816841, 0.04653362, 0.04529797],
       [0.35094893, 0.63207076, 0.01698031],
       [0.93456541, 0.03324022, 0.03219437],
       [0.40615141, 0.04473548, 0.54911311],
       [0.44400595, 0.18601468, 0.36997937],
       [0.13195231, 0.858124  , 0.00992368],
       [0.84231466, 0.08622382, 0.07146152],
       [0.7150574 , 0.14220697, 0.14273563]])

In [29]:
topics = np.argmax(lda_features, axis=1)
body_cleaned = [clean_html(body) for body in posts.Body[posts.Body.notna()].values]
posts_topics = pd.DataFrame({"posts": body_cleaned[:10], "topics":topics})

In [31]:
posts_topics

,posts,topics
0,I've always wanted to start writing (in a tota...,0
1,What kind of story is better suited for each p...,1
2,"I finished my novel, and everyone I've talked ...",0
3,When writing a short-story to highlight a cert...,1
4,"I keep hearing about literary fiction, and how...",0
5,BeginningWriters.com has some good articles fo...,2
6,"Whenever I attempt to write something, I do it...",0
7,If you have a point that you do want to convey...,1
8,I heard a writer talking about pantsing a stor...,0
9,"In the sense you mean, it probably stands for ...",0
